In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd
import re

In [2]:
import sys
sys.path.append('d:/myprojects/project_api_data_orchestrator/src')

%run ../src/project_api_data_orchestrator/core/config.py
%run ../src/project_api_data_orchestrator/db/connection.py

In [3]:

## Scrape Depth Chart Functions
BASE_URL = "https://www.ourlads.com"
DEPTHCHARTS_INDEX = BASE_URL + "/nfldepthcharts/depthcharts.aspx"

# Target positions (including variants for WR)
TARGET_POS = {"QB", "WR", "RB", "TE", "LWR", "RWR", "SWR"}

def get_team_links():
    resp = requests.get(DEPTHCHARTS_INDEX, headers={"User-Agent": "Mozilla/5.0"})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")
    links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("/nfldepthcharts/depthchart/"):
            full = BASE_URL + href
            if full not in links:
                links.append(full)
    return links

def parse_team_depthchart(team_url):
    resp = requests.get(team_url, headers={"User-Agent": "Mozilla/5.0"})
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # Derive team abbreviation
    team_abbrev = None
    depwrapper = soup.find("div", id="ctl00_phContent_DepWrapper")
    if depwrapper:
        for c in depwrapper.get("class", []):
            if c.startswith("dt-"):
                team_abbrev = c[3:]
                break
    if not team_abbrev:
        team_abbrev = team_url.rstrip("/").split("/")[-1].upper()

    records = []
    # Locate the table
    table = depwrapper.find("table", class_="table-bordered") if depwrapper else None
    if table is None:
        table = soup.find("table", class_="table-bordered")
    if table is None:
        return records

    tbody = table.find("tbody") or table.find("tbody", id="ctl00_phContent_dcTBody")
    if tbody is None:
        return records

    for tr in tbody.find_all("tr"):
        tds = tr.find_all("td")
        if len(tds) < 2:
            continue

        pos_raw = tds[0].get_text(strip=True)
        pos = pos_raw.strip()
        pos_norm = "WR" if pos in ("LWR", "RWR", "SWR") else pos

        if pos_norm not in TARGET_POS:
            continue

        tier = 1
        for num_idx in range(1, len(tds), 2):
            player_idx = num_idx + 1
            if player_idx >= len(tds):
                break
            a = tds[player_idx].find("a")
            if a and a.get_text(strip=True):
                player_text = a.get_text(strip=True)
                player_text = re.sub(
                    r"\s+(?:[A-Z]{2}\d{2}|\d{2}/\d|[A-Z]{1,2}/[A-Za-z]{2,3})$",
                    "",
                    player_text,
                    flags=re.IGNORECASE,
                )
                player_clean = player_text.title().strip()
                records.append((player_clean, team_abbrev, pos_norm, tier))
            tier += 1

    return records

def scrape_all_to_dataframe():
    team_links = get_team_links()
    print(f"Found {len(team_links)} team pages.")
    all_records = []
    for link in team_links:
        try:
            recs = parse_team_depthchart(link)
            # print(f"  {link} → {len(recs)} records")
            all_records.extend(recs)
        except Exception as e:
            print(f"Error parsing {link}: {e}")
        time.sleep(1)

    # Deduplicate
    seen = set()
    deduped = []
    for rec in all_records:
        if rec not in seen:
            seen.add(rec)
            deduped.append(rec)

    df = pd.DataFrame(deduped, columns=["Player", "Team", "Position", "Tier"])
    print(f"Scraped {len(df)} unique player-position records.")
    return df

In [ ]:
df_new = scrape_all_to_dataframe()
print(df_new.head())

            Player Team Position  Tier
0   Palmer, Joshua  BUF       WR     1
1  Shavers, Tyrell  BUF       WR     2
2    Coleman, Keon  BUF       WR     1
3   Samuel, Curtis  BUF       WR     2
4   Shakir, Khalil  BUF       WR     1


In [5]:

sorted(df['Team'].unique())

['ARZ',
 'ATL',
 'BAL',
 'BUF',
 'CAR',
 'CHI',
 'CIN',
 'CLE',
 'DAL',
 'DEN',
 'DET',
 'GB',
 'HOU',
 'IND',
 'JAX',
 'KC',
 'LAC',
 'LAR',
 'LV',
 'MIA',
 'MIN',
 'NE',
 'NO',
 'NYG',
 'NYJ',
 'PHI',
 'PIT',
 'SEA',
 'SF',
 'TB',
 'TEN',
 'WAS']

In [11]:

conn = get_connection('nfl_data')

query = "SELECT * FROM public.player_seasons ORDER BY player_season_id ASC"
df_pg = pd.read_sql_query(query, conn)
conn.close()

C:\Users\Tom\AppData\Local\Temp\ipykernel_20312\2036481681.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pg = pd.read_sql_query(query, conn)


In [7]:

## Check if Teams match
sorted(df['Team'].unique()) == sorted(df_pg['abbreviation'].unique())

True

In [12]:
df_pg.head()

,player_season_id,player_id,season_id,tier,notes
0,1,471,1,1,None
1,2,472,1,2,None
2,3,473,1,1,None
3,4,474,1,2,None
4,5,475,1,1,None


In [37]:
conn = get_connection('nfl_data')
cur = conn.cursor()

In [14]:
# === CONFIG ===
SEASON_ID = 1
DELETE_MISSING_PLAYERS = True  # set to False if you only want to remove from player_seasons

In [ ]:
# === Helper functions ===

def get_team_id(team_code):
    cur.execute("SELECT team_id FROM public.teams WHERE abbreviation = %s", (team_code,))
    res = cur.fetchone()
    if res:
        return res[0]

def get_or_create_player(player_name, position, team_code):
    team_id = get_team_id(team_code)
    cur.execute("""
        SELECT player_id FROM public.players 
        WHERE player_name = %s AND team_id = %s
    """, (player_name, team_id))
    res = cur.fetchone()
    if res:
        return res[0]
    cur.execute("""
        INSERT INTO public.players (player_name, position, team_id)
        VALUES (%s, %s, %s)
        RETURNING player_id
    """, (player_name, position, team_id))
    player_id = cur.fetchone()[0]
    conn.commit()
    return player_id

def upsert_player_season(player_id, season_id, tier):
    cur.execute("""
        SELECT player_season_id FROM public.player_seasons
        WHERE player_id = %s AND season_id = %s
    """, (player_id, season_id))
    res = cur.fetchone()
    if res:
        cur.execute("""
            UPDATE public.player_seasons
            SET tier = %s
            WHERE player_season_id = %s
        """, (tier, res[0]))
    else:
        cur.execute("""
            INSERT INTO public.player_seasons (player_id, season_id, tier)
            VALUES (%s, %s, %s)
        """, (player_id, season_id, tier))
    conn.commit()

In [39]:
# === Step 1: Insert or update new data ===
new_player_ids = []
for _, row in df_new.iterrows():
    player_name = row['Player']
    team_code = row['Team']
    position = row['Position']
    tier = row['Tier']

    player_id = get_or_create_player(player_name, position, team_code)
    upsert_player_season(player_id, SEASON_ID, tier)
    new_player_ids.append(player_id)

print("✅ Upserts complete")

(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(4,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(19,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(24,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(3,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(8,)
(26,)
(26,)
(26,)
(26,)
(26,)
(26,)
(26,)
(26,)
(26,)
(26,)
(26,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(13,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(15,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(30,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,)
(10,

In [41]:
# === Step 2: Remove players no longer in list ===
cur.execute("SELECT player_id FROM public.player_seasons WHERE season_id = %s", (SEASON_ID,))
existing_player_ids = [row[0] for row in cur.fetchall()]

players_to_remove = list(set(existing_player_ids) - set(new_player_ids))

if players_to_remove:
    print(f"🗑 Removing {len(players_to_remove)} players from player_seasons...")

    # Remove from player_seasons
    cur.execute("""
        DELETE FROM public.player_seasons
        WHERE season_id = %s AND player_id = ANY(%s)
    """, (SEASON_ID, players_to_remove))
    conn.commit()

    if DELETE_MISSING_PLAYERS:
        # Remove from players if no longer in any season
        cur.execute("""
            DELETE FROM public.players
            WHERE player_id IN (
                SELECT p.player_id FROM public.players p
                LEFT JOIN player_seasons ps ON ps.player_id = p.player_id
                WHERE ps.player_id IS NULL
            )
        """)
        conn.commit()
else:
    print("✅ No players to remove")

print("🏁 Database synchronization complete!")

cur.close()
conn.close()

🗑 Removing 22 players from player_seasons...
🏁 Database synchronization complete!


In [43]:
players_to_remove

[896,
 776,
 651,
 917,
 805,
 678,
 819,
 692,
 822,
 578,
 579,
 580,
 838,
 724,
 736,
 609,
 484,
 492,
 757,
 885,
 888,
 637]